In [1]:
import argparse

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

load nci


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
n_kfold = 1
for dim in target_dim:
    for target_index in np.arange(res.shape[dim]):
        print(dim)
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        true_data_s = pd.DataFrame()
        predict_data_s = pd.DataFrame()
        for fold in range(n_kfold):
            true_data, predict_data = nihgcn_new(
                cell_exprs=exprs,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
            )
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)], ignore_index=True
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)], ignore_index=True
            )

0
epoch:   0 loss:0.705260 auc:0.4680
epoch:  20 loss:0.496013 auc:0.5064
epoch:  40 loss:0.405864 auc:0.5644
epoch:  60 loss:0.361345 auc:0.5351
epoch:  80 loss:0.330091 auc:0.5679
epoch: 100 loss:0.316184 auc:0.5712
epoch: 120 loss:0.304791 auc:0.5769
epoch: 140 loss:0.297101 auc:0.5573
epoch: 160 loss:0.290345 auc:0.5677
epoch: 180 loss:0.287181 auc:0.5577
epoch: 200 loss:0.283710 auc:0.5463
epoch: 220 loss:0.280644 auc:0.5616
epoch: 240 loss:0.277712 auc:0.5538
epoch: 260 loss:0.276374 auc:0.5382
epoch: 280 loss:0.273135 auc:0.5347
epoch: 300 loss:0.271020 auc:0.5346
epoch: 320 loss:0.270131 auc:0.5272
epoch: 340 loss:0.270153 auc:0.5330
epoch: 360 loss:0.265967 auc:0.5331
epoch: 380 loss:0.264377 auc:0.5150
epoch: 400 loss:0.262643 auc:0.5340
epoch: 420 loss:0.261665 auc:0.5187
epoch: 440 loss:0.260486 auc:0.5161
epoch: 460 loss:0.259704 auc:0.5535
epoch: 480 loss:0.257955 auc:0.5394
epoch: 500 loss:0.257235 auc:0.5367
epoch: 520 loss:0.255477 auc:0.5226
epoch: 540 loss:0.254327 a

In [ ]:
true_datas.to_csv(f"new_true_{args.data}.csv")
predict_datas.to_csv(f"new_pred_{args.data}.csv")